In [2]:
# Project 1

In [3]:
import math


def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def check_solution(travel_list, df, speed=30.):
    """
    Check a bank robber algorithm solution
    
    input:
    ------
    travel_list: a list of integers
        Your solution to the bank robber problem
        Bank IDs are row indeces into the df passed
    df: pd.DataFrame
        The algorithm input formatted dataframe 
        (id,x_coordinate,y_coordinate,money,time (hr))
    speed: float
        The km/h speed of travel
    """
    t_remaining = 24.
    score = 0.
    assert len(travel_list) == len(set(travel_list)), (
        "Your travel list must have unique IDs!"
    )
    prev = travel_list[0]
    for e in travel_list:
        row = df.iloc[e]
        score += row['money']
        t_remaining -= row['time (hr)']
        dist = distance(
            row['x_coordinate'], row['y_coordinate'],
            df.iloc[prev]['x_coordinate'],
            df.iloc[prev]['y_coordinate'],
        )
        t_remaining -= dist / speed
        prev = e
    assert t_remaining >= 0, (
        f"Used more than 24h! Time left: {t_remaining}"
    )
    # still gotta get to (0, 0)
    dist = distance(row['x_coordinate'],row['y_coordinate'],0,0)
    final_t = t_remaining - (dist / speed)
    assert final_t >= 0, (
        f"Not enough time to get to helicopter!\n"
        f"Time left after last bank: {t_remaining}\n"
        f"Distance to (0,0) helipad: {dist}\n"
    )
    print(f"Time Remaining: {final_t}")
    return score

In [4]:
import timeit
import pandas as pd
import numpy as np
import math
start_time = timeit.default_timer()

heist = pd.read_csv('C:/Users/Simona/Desktop/Data Analysis/concordia-bootcamp/ds-algorithm-project-1-main/ds-algorithm-project-1-main/data/bank_data.csv')
df = heist.copy()

time = 24
time_left = time
speed = 30 # km/h
banks_robbed = []
money = 0

def distance(x1,y1,x2,y2):
    d = ((x2-x1)**2 + (y2-y1)**2)**0.5
    return d
    
def profit_time(money, time, distance):
    return money/(time*distance)

def next_bank(x1,y1):
    bad_df = df.index.isin(banks_robbed)
    excluding_df = df[~bad_df]
    excluding_money = excluding_df['money'].values
    excluding_time = excluding_df['time (hr)'].values
    excluding_x = excluding_df['x_coordinate'].values
    excluding_y = excluding_df['y_coordinate'].values
    excluding_d = distance(x1,y1,excluding_x,excluding_y)
    profit_T = profit_time(excluding_money, excluding_time, excluding_d)
    max_id = np.where(profit_T==profit_T.max())
    return excluding_df.iloc[int(max_id[0])]

def rob(x1,y1):
    global money
    global banks_robbed
    global time_left
    new_df = next_bank(x1,y1)
    new_x1 = new_df['x_coordinate']
    new_y1 = new_df['y_coordinate']
    time_to_rob = ((math.hypot(new_x1-x1,new_y1-y1)/speed)+new_df['time (hr)']) 
    time_left -= time_to_rob
    money += int(new_df['money'])
    banks_robbed.append(int(new_df['id']))
    if time_left>time_to_rob:
        return rob(new_x1,new_y1)

rob(0,0)
banks_to_rob = banks_robbed[::-1]
print(banks_to_rob)
print(f"\n{money:,d}$")

elapsed = timeit.default_timer() - start_time

[4781, 7769, 9049, 5126, 9062, 523, 7593, 8920, 7625, 8601, 2741, 2346, 5858, 8250, 4627, 4465, 5155, 8661, 9736, 2681, 8206, 2656, 8966, 3297, 9401, 8469, 2327, 7801, 5627, 4560, 2458, 6987, 4345, 5610, 5562, 3516, 757, 2796, 6317, 5622, 4291, 5921, 3450, 9320, 8360, 7265, 8244, 1860, 7650, 1447, 5378, 8908, 1728, 8407, 2827, 5142, 4287, 4605, 8355, 1757, 4789, 4696, 7764, 7701, 2729, 7074, 6740, 7258, 7064, 2344, 4720, 1844, 3025, 6228, 2119, 4293, 6104, 3919, 7531, 1997, 6759, 9100, 9653, 6976, 3507, 8295, 8288, 7869, 1676, 7102, 4742, 5592, 4983, 1058, 8703, 8256, 8286, 2521, 2643, 2331, 2748, 3216, 8525, 3926, 3193, 4877, 6712, 2037, 9130, 9455, 8022, 8579, 5295, 1914, 6468, 299, 4762, 5577, 8006, 6022, 3665, 507, 5836, 5298, 5835, 1707, 3190, 9228, 524, 7595, 1733, 670, 9881, 1372, 951, 2, 58, 433, 209, 5135, 3914, 8562, 444, 3466, 865, 9583, 9378, 2928, 9241, 4725, 7544, 8287, 6097, 8436, 8690, 4757, 4499, 487, 7343, 7560, 5933, 2518, 3803, 8550, 6528, 3798, 4987, 9195, 9546, 36

In [5]:
print(f"Time elapsed: {elapsed} seconds")

Time elapsed: 0.126423400000931 seconds


In [6]:
check_solution(banks_to_rob,df)

Time Remaining: 0.31247804427760484


8937100.0

In [7]:
df

,id,x_coordinate,y_coordinate,money,time (hr)
0,0,2.851925,1.201712,29700,0.273403
1,1,4.150372,-2.654334,6500,0.064040
2,2,-1.494092,-1.230419,89400,0.127458
3,3,1.271326,-0.088520,96100,1.315029
4,4,2.471113,-0.592810,41100,0.164393
...,...,...,...,...,...
9995,9995,-2.472127,4.787304,2100,0.494904
9996,9996,1.034968,-4.321240,54900,0.492866
9997,9997,-0.512720,-2.949173,32200,0.933033
9998,9998,2.626841,-3.482923,5100,0.551312
